# 4 - TRANSFORMACIÓN DE DATOS

## IMPORTAR PAQUETES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

## IMPORTAR LOS DATOS

In [2]:
df = pd.read_pickle('../../02_Datos/03_Trabajo/tablon_analitico.pickle')
df.head()

,mpg,desplazamiento,caballos,peso,aceleracion,1200,500,504,amc,aspen,...,subaru,sw,torino,toyota,vega,volkswagen,volvo,cilindros,any_modelo,origen
0,15.0,350.0,165.0,3693.0,11.5,0,0,0,0,0,...,0,0,0,0,0,0,0,8,70,1
1,16.0,304.0,150.0,3433.0,12.0,0,0,0,1,0,...,0,0,0,0,0,0,0,8,70,1
2,15.0,429.0,198.0,4341.0,10.0,0,1,0,0,0,...,0,0,0,0,0,0,0,8,70,1
3,14.0,454.0,220.0,4354.0,9.0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,70,1
4,14.0,440.0,215.0,4312.0,8.5,0,0,0,0,0,...,0,0,0,0,0,0,0,8,70,1


In [3]:
x = df.drop(columns='mpg').copy()
target = df.mpg.copy().reset_index(drop=True)


## TRANSFORMACIÓN DE CATEGÓRICAS

### ONE HOT ENCODER

#### Variable a aplicar OHE

In [4]:
var_ohe = ['any_modelo','origen']

#### Instanciar

In [5]:
ohe = OneHotEncoder(sparse_output= False, handle_unknown= 'ignore')

#### Entrenar y aplicar

In [6]:
cat_ohe = ohe.fit_transform(x[var_ohe])

In [7]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

#Eliminamos variables

x = x.drop(columns= x[var_ohe])

#### Guardar como dataframe

### ORDINAL ENCODING

In [8]:
df.cilindros.value_counts()

var_oe = ['cilindros']

#Orden de la primera variable
orden_formacion = [2,3,4,5,6,8]

oe = OrdinalEncoder(categories = [orden_formacion],
                    handle_unknown = 'use_encoded_value',
                    unknown_value = -1)

cat_oe = oe.fit_transform(x[var_oe])

#Añadir sufijos a los nombres
nombres_oe = [variable + '_oe' for variable in var_oe]

#Guardar como dataframe
cat_oe = pd.DataFrame(cat_oe, columns = nombres_oe)


Quitamos la variable cilindros del dataframe

In [9]:
x = x.drop(columns='cilindros')

In [10]:
x = pd.concat([x,cat_ohe,cat_oe],axis= 1)

## TRANSFORMACIÓN DE VARIABLES NUMÉRICAS

### CON YEO-JOHNSON

#### Variables a normalizar con Yeo_Johnson

In [11]:
var_yeo = ['desplazamiento','caballos','peso','aceleracion']

#### Instanciar 

In [12]:
yeo = PowerTransformer(method='yeo-johnson')

#### Entrenar y aplicar

In [13]:
num_yeo = yeo.fit_transform(x[var_yeo])

#### Guardar como dataframe

In [14]:
#Añadir sufijos a los nombres
nombres_yeo = [variable + '_yeo' for variable in var_yeo]

#Guardar como dataframe
num_yeo = pd.DataFrame(num_yeo,columns= nombres_yeo)

#### Eliminar y unificar variables

In [15]:
#### Eliminar variables originales
x.drop(columns= var_yeo,inplace= True)

In [16]:
x = pd.concat([x, num_yeo], axis=1)

### RESCALAR VARIABLES

#### Variables a reescalar con Min-Max

In [17]:
var_mms = x.iloc[:,89:].columns

#### Instanciar

In [18]:
mms = MinMaxScaler()

#### Entrenar y aplicar

In [19]:
df_mms = mms.fit_transform(x[var_mms])

#### Guardar como Dataframe

In [20]:
#Añadir sufijos a los nombres

nombres_mms = [variable + '_mms' for variable in var_mms]

df_mms = pd.DataFrame(df_mms, columns= nombres_mms)

#### Eliminar y unificar variables

In [21]:
x.drop(columns= var_mms, inplace= True)

x = pd.concat([x,df_mms,target],axis=1)

## GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

In [22]:
x.to_pickle('../../02_Datos/03_Trabajo/df_tablon.pickle')